In [1]:
#import required packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import re
import numpy as np
from sklearn.metrics import f1_score,classification_report, confusion_matrix,precision_score, recall_score
from tensorflow import keras

In [19]:
#post data
data = pd.read_csv('input_data/Post_ModelPredictions_ManualChecking.csv')
df = data.copy()
df.head()

,post,prefLabel,compulsion_ml,obs-com_ml,obsession_ml,Document_Keywords,compulsion_bert,obs-com_bert,obsession_bert,obsession_first_checker,obs-com_first_checker,compulsion_first_checker
0,"I grew up with obsessive thoughts, or more apt...","['site', 'compulsion', 'obsession', 'rash', 'p...",1,0,1,"['grow', 'obsessive_thought', 'aptly', 'intrus...",0.0,0.0,1.0,1.0,1.0,1.0
1,I have been thinking so much about Covid becau...,"['singing', 'washing hands', 'anxiety', 'anxie...",0,0,1,"['think', 'covid', 'job', 'concern', 'encourag...",0.0,0.0,1.0,0.0,0.0,1.0
2,"I fed up, I send all day checking excessive ch...","['OCD', 'checking', 'Thought']",0,0,1,"['send', 'day', 'check', 'excessive', 'checkin...",0.0,0.0,1.0,0.0,0.0,1.0
3,I’ve noticed that I’ve been washing my hands m...,"['washing hands', 'hope', 'depression', 'OCD']",1,0,1,"['notice', 'washing_hand', 'thing', 'trigger',...",0.0,0.0,1.0,1.0,0.0,0.0
4,This sounds so stupid\nAt the moment I want to...,['Thought'],0,0,1,"['sounds_stupid', 'moment', 'download', 'windo...",0.0,0.0,1.0,1.0,1.0,1.0


In [5]:
import re
#define the process of text cleaning
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)
#Clean Text
def clean_text(data):
    # convert catacter to lowercase
    data['clean_text']=data['post'].str.lower()
    #remove URLS
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"http\S+", "", elem))
    #remove ponctuation
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"[^\w\s]", "", elem))
    #remove
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'/n',"",elem))
    #remove degits
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\d+',"",elem))
    #remove emojis
    data['clean_text'] = data['clean_text'].apply(lambda elem:deEmojify(elem))
    #remove multiple spaces
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+'," ",elem))
    #remove single caracter
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+[a-zA-Z]\s+'," ",elem))
    return data

In [6]:
#apply the process of cleaning for the train and test data
df = clean_text(df)

In [7]:
#for the text pre-processing (text cleaning)
import nltk
nltk.download('punkt')
import re #regular expression
import string
from nltk.tokenize import word_tokenize # word tokenization
from nltk.stem import PorterStemmer # word stemming

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def remove_stop_words(text):
  stopW=stopwords.words('english')
  s=""
  for i in text.split():
    if i not in stopW:
        s=s+i+" "
  return s

In [10]:
df['clean_text']=df['clean_text'].apply(lambda x:remove_stop_words(x))

In [11]:
def stem(ch):
  stem = PorterStemmer()
  return " ".join([stem.stem(i) for i in ch.split()])

#apply the stem function to each row in the dataframe
df['clean_text'] = df['clean_text'].apply(lambda x:stem(x))

In [14]:
X_test = df['clean_text']

In [13]:
import pickle
with open('model/lstm_tockenizer.pkl', 'rb') as f:
    vect = pickle.load(f)
vocab_size = len(vect.word_index) + 1
print(vocab_size)

9464


In [15]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LEN = 100
encoded_docs_test =  vect.texts_to_sequences(X_test)
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=MAX_LEN, padding='post')
len(padded_docs_test)

23674

In [16]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

In [17]:
model = Sequential()
# Configuring the parameters
model.add(Embedding(vocab_size, output_dim=16, input_length=MAX_LEN))
model.add(LSTM(16, return_sequences=True))  
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(16))
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(2, activation='sigmoid'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           151424    
                                                                 
 lstm (LSTM)                 (None, 100, 16)           2112      
                                                                 
 dropout (Dropout)           (None, 100, 16)           0         
                                                                 
 lstm_1 (LSTM)               (None, 16)                2112      
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 2)                 34        
                                                                 
Total params: 155,682
Trainable params: 155,682
Non-trai

In [18]:
model.load_weights('model/lstm_model.h5')

In [20]:
predictions=model.predict([padded_docs_test])

740/740 [==============================] - 6s 7ms/step


In [21]:
len(predictions)

23674

In [22]:
data['compulsion_lstm_pred'] = predictions[:, 0]
data['obsession_lstm_pred'] = predictions[:, 1]

In [23]:
data.to_csv('input_data/Post_ModelPredictions_lstm_ManualChecking.csv')